In [1]:
import re
import numpy as np
import urllib.request
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [3]:
# mapping: soundclassid -> spiece
mapping = {
    "295": "NaN\tNaN",
    "363": "Rhfo\t7",
    "366": "Bada\t6",
    "449": "Rhmo\t8",
    "496": "Mupu\t1",
    "497": "Myla\t5",
    "498": "Piab\t2",
    "499": "Plta\t6",
    "500": "Scku\t2",
    "501": "Vesi\t3",
    "502": "NaN\tNaN",
    "503": "Kefu\t10",
    "504": "Myse\t5",
    "505": "Myfr\t5",
    "506": "Mugr\t1",
    "507": "NaN\t4",
    "508": "NaN\t4",
    "509": "Tain\t11",
    "510": "NaN\tNaN",
    "511": "Hais\t1",
    "512": "NaN\tNaN",
    "513": "NaN\t1",
    "514": "NaN\t1",
    "515": "NaN\tNaN",
    "516": "Sula\t5",
    "517": "Myfi\t5",
    "518": "Myso\t5",
    "519": "NaN\tNaN",
    "520": "Epse\t3",
    "521": "Hiar\t9",
    "522": "Mifu\t2",
    "523": "Myfo\t5",
    "524": "Myru\t5",
    "525": "Nypl\t3",
    "526": "NaN\tNaN",
    "546": "Haha\t1",
}

In [4]:
wanted_id = []
wanted_audio_id = []
wanted_file_name = []
seq_segs = []
# with open("as_label.txt", "r") as f:
with open("0627_2340", "r") as f:    
    lines = f.readlines()
    for line in lines:
        data = line.split('\t')
        if data[11] == 'v':
            wanted_id += [data[0]]
            wanted_audio_id += [data[10]]
            wanted_file_name += [data[1]]
            seq_segs += [data[13]]

In [5]:
page = urllib.request.urlopen('https://silic.tbn.org.tw/api/labels?fbclid=IwAR08a16vXmMWAI3w4follrp4zuOipu7i6866OUE8_LOz_nVFIRsS6JnMOsw')
content = page.read().decode(page.headers.get_content_charset())

In [6]:
split_text = re.split('\t|\n', content)

In [7]:
# info
write_path = '0627_2340_table.txt'
num_elements = 14

# rearrange text into a list of lists
# each of list contains 14 elements
num_rows = int(len(split_text) / num_elements)
# reshape 
reshape_text = np.reshape(np.array(split_text), [num_rows, num_elements])

# audioid localfilename source sourceid sourcefilename 
# labelid labelannotationid time_begin time_end freq_low 
# freq_high soundclassid aiscore score

cursor = 0
counter = 1

with open(write_path, 'w') as f:
    for i in range(1, num_rows-1):
        data = list(reshape_text[i])
        
        # only put lines with wanted id
        if data[0] in wanted_audio_id:
            info = ""
            
            index = wanted_audio_id.index(data[0])
            file_number = str(wanted_id[index])
            file_name = wanted_file_name[index]
            seq_seg = seq_segs[index]
            
            # process
            times = [str(float(time)/1000.) for time in data[7:9]]
            try:
                spiece = mapping[str(data[11])]
            except:
                spiece = 'NULL\tNULL'

            # add number for duplicated audio id
            current_id = data[0]
            if data[0] == cursor:
                counter += 1
            else:
                cursor = data[0]
                counter = 1
            # seq num
            if seq_seg == '0':
                seq_num = '1'
                info += "seq note: 0"
            else:
                seg_split = seq_seg.split('/')
                if all([is_float(ss) for ss in seg_split]):
                    # compute seq num
                    seq_arr = [float(ss) for ss in seg_split] + [float(times[0])]
                    seq_arr.sort()
                    seq_num = str(seq_arr.index(float(times[0])))
                    info += "seq note: {}, times begin: {}, seq num: {}".format(seq_seg, times[0], seq_num)
                else:
                    # return origianl seq note
                    seq_num = seq_seg
                    info += "no seq num"
            
            full_str = [file_number] + [file_name] + [str(counter)] + times + data[9:11] + [seq_num] + [spiece] + [info]
            f.write('\t'.join(full_str) + ('' if i == num_rows-1 else '\n'))

In [8]:
# sort lines based on file number
with open(write_path, 'r') as f:
    lines = f.readlines()
    lines.sort(key=lambda l: int(l.split('\t')[0]))

with open(write_path, 'w') as f:
    for line in lines:
        f.write(line)